In [1]:
import numpy as np
import h5py
import matplotlib.pyplot as plt

%matplotlib inline
plt.rcParams['figure.figsize'] = (5.0, 4.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

%load_ext autoreload
%autoreload 2

np.random.seed(1)

### Passo a Passo

O objetivo é criar blocos de instrução de uma rede neural

### Zero page

Colocar zeros envolta das imagens para:

- Ajuda na hora de transforma a imagem para não perde qualidade das bordas na hora de transforma ela em um tamanho menor, pois, a borda não tem outros pixels do lado

- Ajuda a manter todas as informações necessárias

In [2]:
def zero_pad(X, pad):

  X_pad = np.pad(X, ((0,0), (pad,pad), (pad,pad), (0,0)), mode='constant', contant_values= (0,0))

### Passo unico de Convolução

Cria um filtro para realizar um passo do convolução:

- Onde pega o volume da entrada
- Aplica o Filtro
- Sai um outro volume de entrada


Nessa primeira etapa vamos utilizar apenas um filtro e aplicar em uma saida

In [3]:
def conv_single_step(a_slice_prev, W, b):

    s = np.multiply(a_slice_prev, W)
    z = np.sum([s])
    Z = np.sum([z, np.squeeze(b)])

    return Z

### Redes Neuronais Convolucionais - Passe para a frente

É passado um filtro para transforma a imagem em 2d e depois transforma-la em 3d

In [4]:
def conv_foward(A_prev, W, b, hparameters):
 # Extrai as dimensões das ativações de entrada (A_prev), pesos do filtro (W), vieses (b) e hiperparâmetros (hparameters)
# Calcula as dimensões do volume de saída (Z) com base no tamanho de entrada, tamanho do filtro, passo e preenchimento (padding)
    
   # Extrai as dimensões das ativações de entrada (A_prev)
  (m, n_H_prev, n_W_prev, n_C_prev) = A_prev.shape

  # Extrai as dimensões do filtro (W)
  (f, f, n_C_prev, n_C) = W.shape

  stride = hparameters["stride"] # Extrai o passo
  pad = hparameters["pad"] # Extrai o preenchimento 

  # Calcula a altura e largura do volume de saída (Z) usando a fórmula de convolução
  n_H = int((n_H_prev + (2 * pad)-f) / stride) + 1
  n_W = int ((n_W_prev + (2 *pad)-f) / stride) + 1

  # Inicializa o volume de saída (Z) com zeros  
  Z = np.zeros((m, n_H, n_W, n_C))

  # Realiza o preenchimento (padding) zero nas ativações de entrada
  A_prev_pad = zero_pad(A_prev, pad)

 # Loop pelos exemplos no mini-lote (m) e aplica a operação de convolução deslizando o filtro sobre o volume de entrada
  for i in range(m):  
    a_prev_pad = A_prev_pad[i]         # Seleciona o i-ésimo exemplo com preenchimento
    
    # Loop pela altura do volume de saída (Z)
    for h in range(n_H):        

      
      vert_start = stride * h   # Define o índice inicial vertical
      vert_end = vert_start + f  # Define o índice final vertical

      # Loop pela largura do volume de saída (Z)
      for w in range(n_W):

      
        horiz_start = stride * w # Define o índice inicial horizontal
        horiz_end = horiz_start + f # Define o índice final horizontal


         # Loop pelos canais do volume de saída (Z)
        for c in range(n_C): 

          
          # Extrai a "fatia" correspondente do volume de entrada
          a_slice_prev = a_prev_pad[vert_start:vert_end, horiz_start:horiz_end,:]

          # Extrai os pesos e vieses do filtro
          weight = W[:, :, :, c]
          biases = b[:, :, :, c]

          # Realiza uma única etapa da convolução usando a função conv_single_step
          Z[i, h, w, c] = conv_single_step(a_slice_prev, weight, biases)

  cache = (A_prev, W, b, hparameters)

  return Z, cache
          

  